### Parse KMZ 
http://programmingadvent.blogspot.com/2013/06/kmzkml-file-parsing-with-python.html

In [1]:
from zipfile import ZipFile
from bs4 import BeautifulSoup

kmz_filename = 'al182012_best_track.kmz'
kml_filename = 'al182012.kml'

archive = ZipFile(kmz_filename, 'r')
with archive.open(kml_filename, 'r') as content_file:
    content = content_file.read()
    
soup = BeautifulSoup(content, 'html.parser')

#print (soup.prettify())

In [2]:
path = []
for pm in soup.find_all('placemark'):
    timestamp = pm.find('dtg').get_text().encode('utf-8')
    yr = pm.find('atcfdtg').get_text().encode('utf-8')[:4]
    timestamp = yr + ' ' + timestamp
    lat = pm.find('lat').get_text().encode('utf-8')
    lon = pm.find('lon').get_text().encode('utf-8')
    path.append([timestamp, lat, lon])
    
#path

### Save data in a Pandas dataframe

In [3]:
import numpy as np
import pandas as pd

def format_kml_utc_date_string(dstr):
    dstr = dstr.replace('UTC','')
    dstr = dstr.split()
    dstr_t = dstr[1]
    dstr_t = ':'.join([dstr_t[i:i+2] for i in xrange(0,len(dstr_t),2)])
    dstr[1] = dstr_t
    return ' '.join([dstr[0], dstr[2], dstr[3], dstr[1]])

In [4]:
path_df = pd.DataFrame(path, columns=['timestamp', 'lat', 'lon'])
path_df['lat'] = path_df['lat'].astype('float64')
path_df['lon'] = path_df['lon'].astype('float64')
path_df['timestamp'] = path_df['timestamp'].map(format_kml_utc_date_string)
path_df['timestamp'] = pd.to_datetime(path_df['timestamp'], format='%Y %b %d %H:%M')
path_df.head()

,timestamp,lat,lon
0,2012-10-21 18:00:00,14.3,-77.4
1,2012-10-22 00:00:00,13.9,-77.8
2,2012-10-22 06:00:00,13.5,-78.2
3,2012-10-22 12:00:00,13.1,-78.6
4,2012-10-22 18:00:00,12.7,-78.7


In [5]:
from bokeh.io import output_notebook, show
output_notebook()

from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import WMTSTileSource
from bokeh.plotting import Figure, show, output_file

from bokeh.models import HoverTool, BoxSelectTool

#import matplotlib as mpl
#import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import numpy as np

Loading BokehJS ...

In [6]:
import pandas as pd
import numpy as np

def wgs84_to_web_mercator(lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    x = lon * (k * np.pi/180.0)
    y = np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x, y

#df_test = pd.DataFrame(dict(name=["Austin","NYC"],lon=[-97.7431,-74.0059],lat=[30.2672,40.7128]))
#x, y = wgs84_to_web_mercator(df_test['lon'].values, df_test['lat'].values)
#df_test['x'], df_test['y'] = x, y
#df_test

In [11]:
#def plot_geo_distbn_points_bokeh(lon_min, lon_max, lat_min, lat_max, lon, lat, timestamp):
def plot_geo_distbn_points_bokeh(path_df):

    # Min and max lat-lon for whole dataset
    lat, lon = path_df['lat'].values, path_df['lon'].values
    lon_min, lon_max = path_df['lon'].values.min(), path_df['lon'].values.max()
    lat_min, lat_max = path_df['lat'].values.min(), path_df['lat'].values.max()


    # save to a html file
    output_file('sandy_path_bokeh.html',title='Contour Plot')

    # Define our longitude and latitude points
    x,y = wgs84_to_web_mercator(lon, lat)
    
    source = ColumnDataSource(
        data=dict(
            x=list(x),
            y=list(y),
            desc=path_df['timestamp'].map(lambda x: x.strftime('%a %b %d %Y %H:%M')),
                 #http://stackoverflow.com/questions/35128934/bokeh-hover-tool-format-date-variable-in-custom-html
        )
    )
    
    hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            #("(x,y)", "($x, $y)"),
            ("Time", "@desc"),
        ]
    )        
    
    x_range, y_range = wgs84_to_web_mercator(np.array([lon_min, lon_max]), np.array([lat_min, lat_max]))
        
    #fig = figure(tools='pan, wheel_zoom', x_range=tuple(x_range), y_range=tuple(y_range))
    fig = figure(x_range=tuple(x_range), y_range=tuple(y_range), tools="hover,save,pan,box_zoom,wheel_zoom,reset",
                 plot_width=300, plot_height=450)
                 #border_fill="#aa8888", plot_width=500, plot_height=500, border=100)
    fig.axis.visible = False

    # WMTS tile sources. See http://geo.holoviews.org/Working_with_Bokeh.html
    url = 'http://a.basemaps.cartocdn.com/light_all/{Z}/{X}/{Y}.png'
    #url = 'http://a.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
    #attribution = "Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL"
    fig.add_tile(WMTSTileSource(url=url))#, attribution=attribution))
    
    #r = fig.circle(x=x, y=y, fill_color="red", fill_alpha = .8, line_color=None)
    fig.circle('x', 'y', fill_color='red', line_color='red', 
               fill_alpha=.5, line_alpha=.5,
               size=10, 
               source=source)

    #glyph = r.glyph
    #glyph.size = 50
    
    fig.select_one(HoverTool).tooltips = [
        #("index", "$index"),
        #("(x,y)", "($x, $y)"),
        ("Time", "@desc"),
    ]
    
    show(fig)

In [13]:
plot_geo_distbn_points_bokeh(path_df)

INFO:bokeh.core.state:Session output file 'sandy_path_bokeh.html' already exists, will be overwritten.


KeyboardInterrupt: 